<a href="https://colab.research.google.com/github/Beloveddream/RS/blob/main/Product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
#载入数据
train_data = pd.read_csv('/content/gdrive/My Drive/Product/sant/train.csv')
test_data = pd.read_csv('/content/gdrive/My Drive/Product/sant/test.csv')
train_data['origin'] = 'train'
test_data['origin'] = 'test'
data = pd.concat([train_data, test_data], ignore_index=True, sort=False)
#重命名列名
data.rename(columns = {'fecha_dato':'date','ncodpers':'user_id','ind_empleado':'employee_type',
                        'pais_residencia':'user_country','sexo':'sex','fecha_alta':'register_date',
                        'ind_nuevo':'new_user_type','antiguedad':'user_seniority','indrel':'user_type',
                        'ult_fec_cli_1t':'last_date_as_primary_user','indrel_1mes':'user_type_at_begin',
                        'tiprel_1mes':'user_relation_at_begin','indresi':'live_country_type','indext':'born_country_type',
                        'conyuemp':'employee_spouse_type','canal_entrada':'user_join_channel','indfall':'alive_type',
                        'tipodom':'address_type','cod_prov':'province_code','nomprov':'province_name',
                        'ind_actividad_cliente':'activity_type','renta':'user_income','segmento':'user_segmentation',
                        'ind_ahor_fin_ult1':'item_1','ind_aval_fin_ult1':'item_2','ind_cco_fin_ult1':'item_3',
                        'ind_cder_fin_ult1':'item_4','ind_cno_fin_ult1':'item_5','ind_ctju_fin_ult1':'item_6',
                        'ind_ctma_fin_ult1':'item_7','ind_ctop_fin_ult1':'item_8','ind_ctpp_fin_ult1':'item_9',
                        'ind_deco_fin_ult1':'item_10','ind_deme_fin_ult1':'item_11','ind_dela_fin_ult1':'item_12',
                        'ind_ecue_fin_ult1':'item_13','ind_fond_fin_ult1':'item_14','ind_hip_fin_ult1':'item_15',
                        'ind_plan_fin_ult1':'item_16','ind_pres_fin_ult1':'item_17','ind_reca_fin_ult1':'item_18',
                        'ind_tjcr_fin_ult1':'item_19','ind_valo_fin_ult1':'item_20','ind_viv_fin_ult1':'item_21',
                        'ind_nomina_ult1':'item_22','ind_nom_pens_ult1':'item_23','ind_recibo_ult1':'item_24'},inplace = True)
#去除一些相关性不太大的特征列
data = data.dropna(axis=0,subset = ['register_date'])
data = data.drop('employee_spouse_type',axis = 1)
data = data.drop('last_date_as_primary_user',axis = 1)
data = data.drop('province_name',axis = 1)
data = data.drop('address_type',axis = 1)
data = data.drop('employee_type',axis = 1)
#对特征列的空值进行补全
data['user_type'] = data['user_type'].replace(np.nan, 2)
data['province_code'] = data['province_code'].replace(np.nan, 99)
data['activity_type'] = data['activity_type'].replace(np.nan, 2)
data['user_country'] = data['user_country'].replace(np.nan, 'unknown')
data['sex'] = data['sex'].replace(np.nan,'unknown')
data['user_seniority'] = data['user_seniority'].replace(np.nan, 0)
data['user_seniority'] = data['user_seniority'].replace('     NA', 0)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('1.0', 1)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('1', 1)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('3.0', 3)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('3', 3)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('2.0', 2)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('2', 2)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('4.0', 4)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('4', 4)
data['user_type_at_begin'] = data['user_type_at_begin'].replace('P', 5)
data['user_type_at_begin'] = data['user_type_at_begin'].replace(np.nan, 0)
data['user_relation_at_begin'] = data['user_relation_at_begin'].replace(np.nan, 'unknown')
data['live_country_type'] = data['live_country_type'].replace(np.nan, 'unknown')
data['born_country_type'] = data['born_country_type'].replace(np.nan, 'unknown')
data['user_join_channel'] = data['user_join_channel'].replace(np.nan, 'unknown')
data = data.drop('alive_type',axis = 1)
data['user_segmentation'] = data['user_segmentation'].replace(np.nan, 'unknown')
#增加用户使用时长的特征
data['date']=pd.to_datetime(data['date'],format='%Y-%m-%d')
data['register_date']=pd.to_datetime(data['register_date'],format='%Y-%m-%d')
data['use_time'] = (data['date'] - data['register_date']).dt.days
data = data.drop('register_date',axis =1)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#提取用户信息特征准备通过xgboost对用户收入进行补全
user_data = data[['user_id','date','user_country','sex','age',
                'user_seniority','user_type','live_country_type','born_country_type',
                'user_join_channel','province_code','activity_type','user_income',
                'user_segmentation','use_time']]
user_data = user_data.sort_values(by=['user_id','date'])
user_data.drop_duplicates(subset = ['user_id'],keep='last',inplace=True)
user_data = user_data.drop('date',axis = 1)
user_data = user_data.reset_index(drop = True)

In [6]:
print(user_data)

       user_id user_country sex  ... user_income  user_segmentation  use_time
0        16074           ES   V  ...    53478.18  02 - PARTICULARES      2529
1        16095           ES   V  ...         NaN           01 - TOP      7766
2        16117           ES   V  ...   465589.68  02 - PARTICULARES      7765
3        16181           ES   H  ...   134978.88  02 - PARTICULARES      7378
4        16308           ES   V  ...   175988.07  02 - PARTICULARES      7565
...        ...          ...  ..  ...         ...                ...       ...
39692  1548081           ES   H  ...         NaN  02 - PARTICULARES        30
39693  1548083           ES   V  ...         NaN            unknown         0
39694  1548094           ES   H  ...         NaN            unknown        -1
39695  1548100           ES   H  ...         NaN  02 - PARTICULARES        29
39696  1548191           ES   H  ...         NaN            unknown        -1

[39697 rows x 14 columns]


In [7]:
#对用户信息特征列进行labelencoder
user_data1 = user_data[['user_id','sex','age','province_code','user_segmentation','user_income']]
from sklearn.preprocessing import LabelEncoder
lbe_sex = LabelEncoder()
user_data1['sex'] = lbe_sex.fit_transform(user_data1['sex'])
lbe_province_code = LabelEncoder()
user_data1['province_code'] = lbe_province_code.fit_transform(user_data1['province_code'])
lbe_user_segmentation = LabelEncoder()
user_data1['user_segmentation'] = lbe_user_segmentation.fit_transform(user_data1['user_segmentation'])
user_data1['age'] = user_data1['age'].astype('int16')
print(user_data1)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1     53478.18
1        16095    1   74             27                  0          NaN
2        16117    1   57             27                  1    465589.68
3        16181    0   77             27                  1    134978.88
4        16308    1   47             27                  1    175988.07
...        ...  ...  ...            ...                ...          ...
39692  1548081    0   44             20                  1          NaN
39693  1548083    1   22              6                  3          NaN
39694  1548094    0   56              2                  3          NaN
39695  1548100    0   54             28                  1          NaN
39696  1548191    0   70             27                  3          NaN

[39697 rows x 6 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [8]:
#将存在收入数据的用户信息作为训练集，不存在收入数据的用户信息作为预测集
user_data_train = user_data1.dropna()
user_data_train['user_income'] = user_data_train['user_income'].astype('int64')
user_data_predict = user_data1[user_data1.user_income.isnull()]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
print(user_data_train)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1        53478
2        16117    1   57             27                  1       465589
3        16181    0   77             27                  1       134978
4        16308    1   47             27                  1       175988
5        16339    1   55             27                  0       197736
...        ...  ...  ...            ...                ...          ...
36184  1454440    0   24             27                  2        85797
36185  1454443    1   21             26                  2        96142
36186  1454499    1   20             27                  1       194682
36187  1454504    0   20             14                  2        54385
36188  1454511    1   28             14                  2       202631

[29850 rows x 6 columns]


In [10]:
print(user_data_predict)

       user_id  sex  age  province_code  user_segmentation  user_income
1        16095    1   74             27                  0          NaN
32       17453    0   75             27                  1          NaN
41       17701    1   53             27                  1          NaN
53       18024    1   60             27                  1          NaN
58       18288    1   70             38                  1          NaN
...        ...  ...  ...            ...                ...          ...
39692  1548081    0   44             20                  1          NaN
39693  1548083    1   22              6                  3          NaN
39694  1548094    0   56              2                  3          NaN
39695  1548100    0   54             28                  1          NaN
39696  1548191    0   70             27                  3          NaN

[9847 rows x 6 columns]


In [11]:
user_data_predict = user_data_predict.drop('user_income',axis =1)

In [12]:
#将训练集中收入偏差均值3个标准差以外的收入数据进行剔除
user_income_mean = user_data_train['user_income'].mean()
print(user_income_mean)
user_income_std = user_data_train['user_income'].std()
print(user_income_std)
user_income_max = int(user_income_mean + 3 * user_income_std)
print(user_income_max)

136586.759798995
320021.55237731023
1096651


In [13]:
user_data_train1 = user_data_train[user_data_train.user_income < user_income_max]
print(user_data_train1)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1        53478
2        16117    1   57             27                  1       465589
3        16181    0   77             27                  1       134978
4        16308    1   47             27                  1       175988
5        16339    1   55             27                  0       197736
...        ...  ...  ...            ...                ...          ...
36184  1454440    0   24             27                  2        85797
36185  1454443    1   21             26                  2        96142
36186  1454499    1   20             27                  1       194682
36187  1454504    0   20             14                  2        54385
36188  1454511    1   28             14                  2       202631

[29786 rows x 6 columns]


In [14]:
#进行xgboost预测
import xgboost as xgb
from sklearn.model_selection import train_test_split

param = {'boosting_type':'gbdt',
                         'objective' : 'reg:linear', 
                         'eval_metric' : 'rmse',
                         'eta' : 0.01,
                         'max_depth' : 10,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         #'alpha': 0.6,
                         'lambda': 2,
        }
user_data_predict1 = xgb.DMatrix(user_data_predict[['sex','age','province_code','user_segmentation']])
#for name in col_name:
X_train, X_valid, y_train, y_valid = train_test_split(user_data_train1.drop(['user_income','user_id'],axis=1), user_data_train1['user_income']
                                                      , test_size=0.2, random_state=2020)

train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_valid, label=y_valid)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)

predict = model.predict(user_data_predict1)

[14:01:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:162561	valid-rmse:163331
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 200 rounds.
[25]	train-rmse:139879	valid-rmse:140810
[50]	train-rmse:123642	valid-rmse:124777
[75]	train-rmse:112524	valid-rmse:113934
[100]	train-rmse:105082	valid-rmse:106734
[125]	train-rmse:100271	valid-rmse:102123
[150]	train-rmse:97076.2	valid-rmse:99134.4
[175]	train-rmse:94956.1	valid-rmse:97235.3
[200]	train-rmse:93585.6	valid-rmse:96055.6
[225]	train-rmse:92659.8	valid-rmse:95342
[250]	train-rmse:92065.2	valid-rmse:94890.7
[275]	train-rmse:91642.2	valid-rmse:94621.5
[300]	train-rmse:91325.2	valid-rmse:94476.2
[325]	train-rmse:91119.5	valid-rmse:94388.1
[350]	train-rmse:90928.2	valid-rmse:94356.2
[375]	train-rmse:90797.5	valid-rmse:94347
[400]	train-rmse:90678.9	valid-rmse:94353.1


In [15]:
user_data_predict['user_income'] = predict

In [16]:
print(user_data_predict)

       user_id  sex  age  province_code  user_segmentation    user_income
1        16095    1   74             27                  0  221232.640625
32       17453    0   75             27                  1  154347.875000
41       17701    1   53             27                  1  165267.593750
53       18024    1   60             27                  1  167669.328125
58       18288    1   70             38                  1  134357.640625
...        ...  ...  ...            ...                ...            ...
39692  1548081    0   44             20                  1   82145.382812
39693  1548083    1   22              6                  3  157018.937500
39694  1548094    0   56              2                  3   76976.218750
39695  1548100    0   54             28                  1   97123.851562
39696  1548191    0   70             27                  3  100967.968750

[9847 rows x 6 columns]


In [17]:
user_data_predict['user_income'] = user_data_predict['user_income'].astype('int64')

In [18]:
print(user_data_train)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1        53478
2        16117    1   57             27                  1       465589
3        16181    0   77             27                  1       134978
4        16308    1   47             27                  1       175988
5        16339    1   55             27                  0       197736
...        ...  ...  ...            ...                ...          ...
36184  1454440    0   24             27                  2        85797
36185  1454443    1   21             26                  2        96142
36186  1454499    1   20             27                  1       194682
36187  1454504    0   20             14                  2        54385
36188  1454511    1   28             14                  2       202631

[29850 rows x 6 columns]


In [19]:
print(user_data_predict)

       user_id  sex  age  province_code  user_segmentation  user_income
1        16095    1   74             27                  0       221232
32       17453    0   75             27                  1       154347
41       17701    1   53             27                  1       165267
53       18024    1   60             27                  1       167669
58       18288    1   70             38                  1       134357
...        ...  ...  ...            ...                ...          ...
39692  1548081    0   44             20                  1        82145
39693  1548083    1   22              6                  3       157018
39694  1548094    0   56              2                  3        76976
39695  1548100    0   54             28                  1        97123
39696  1548191    0   70             27                  3       100967

[9847 rows x 6 columns]


In [20]:
user_data_all = pd.concat([user_data_train,user_data_predict],ignore_index = True)

In [21]:
print(user_data_all)

       user_id  sex  age  province_code  user_segmentation  user_income
0        16074    1   47             27                  1        53478
1        16117    1   57             27                  1       465589
2        16181    0   77             27                  1       134978
3        16308    1   47             27                  1       175988
4        16339    1   55             27                  0       197736
...        ...  ...  ...            ...                ...          ...
39692  1548081    0   44             20                  1        82145
39693  1548083    1   22              6                  3       157018
39694  1548094    0   56              2                  3        76976
39695  1548100    0   54             28                  1        97123
39696  1548191    0   70             27                  3       100967

[39697 rows x 6 columns]


In [22]:
print(data)

             date  user_id user_country sex  ... item_23  item_24 origin  use_time
0      2015-01-28  1050676           ES   V  ...     0.0      0.0  train       901
1      2015-01-28  1050662           ES   V  ...     0.0      0.0  train       901
2      2015-01-28  1050652           ES   H  ...     0.0      1.0  train       901
3      2015-01-28  1050489           ES   H  ...     0.0      0.0  train       901
4      2015-01-28  1050490           ES   V  ...     0.0      0.0  train       901
...           ...      ...          ...  ..  ...     ...      ...    ...       ...
562927 2016-05-28  1166847           ES   V  ...     NaN      NaN   test      1018
562928 2016-05-28  1166605           ES   V  ...     NaN      NaN   test      1018
562929 2016-05-28  1166829           ES   V  ...     NaN      NaN   test      1018
562930 2016-05-28  1166827           ES   H  ...     NaN      NaN   test      1018
562931 2016-05-28  1166781           ES   H  ...     NaN      NaN   test      1018

[56

In [23]:
user_data_all = user_data_all.drop(['sex','age','province_code','user_segmentation'],axis =1)
print(user_data_all)

       user_id  user_income
0        16074        53478
1        16117       465589
2        16181       134978
3        16308       175988
4        16339       197736
...        ...          ...
39692  1548081        82145
39693  1548083       157018
39694  1548094        76976
39695  1548100        97123
39696  1548191       100967

[39697 rows x 2 columns]


In [24]:
#将预测的收入数据填补进原有的数据集中
data = data.drop('user_income',axis=1)
data = pd.merge(data,user_data_all,how='left')

In [25]:
print(data)

             date  user_id user_country  ... origin use_time  user_income
0      2015-01-28  1050676           ES  ...  train      901        90431
1      2015-01-28  1050662           ES  ...  train      901       122456
2      2015-01-28  1050652           ES  ...  train      901       149099
3      2015-01-28  1050489           ES  ...  train      901       158207
4      2015-01-28  1050490           ES  ...  train      901        68574
...           ...      ...          ...  ...    ...      ...          ...
561781 2016-05-28  1166847           ES  ...   test     1018        88937
561782 2016-05-28  1166605           ES  ...   test     1018        92776
561783 2016-05-28  1166829           ES  ...   test     1018        96488
561784 2016-05-28  1166827           ES  ...   test     1018        30303
561785 2016-05-28  1166781           ES  ...   test     1018        66748

[561786 rows x 43 columns]


In [27]:
train_data_new = data[data.origin == 'train']
test_data_new = data[data.origin == 'test']
train_data_new.to_csv('/content/gdrive/My Drive/Product/sant/train_update.csv',index = 0)
test_data_new.to_csv('/content/gdrive/My Drive/Product/sant/test_update.csv',index = 0)